In [1]:
from scipy.spatial import cKDTree
import pyfits, mechanize, bz2, os
from matplotlib import pyplot as pl
from numpy import *
from scipy.optimize import leastsq
import astropy.stats as sta
from scipy import ndimage
from glob import glob
import pandas as pd  #為了讀csv檔。csv檔是以逗點分割的檔案。
import requests
from bs4 import BeautifulSoup
from astropy.io import ascii
import concurrent.futures
from time import sleep
from astropy.table import Table
import math as m
import gzip
import patoolib
import glob

In [2]:
f = open('url.txt', 'r')
x = f.readlines()

In [3]:
x[0][0:-1]

'http://dr2.lamost.org/sas/fits/HD110609N070823M01/spec-56270-HD110609N070823M01_sp07-075.fits.gz'

In [4]:
len(x[5][0:-1])

91

In [5]:
x[5][0:-1][11:15]

'lamo'

In [6]:
for a in range(len(x[0][0:-1])):
    if x[0][0:-1][a:a+4] == 'spec':
        filename = x[0][0:-1][a:]

In [7]:
filename

'spec-56270-HD110609N070823M01_sp07-075.fits.gz'

In [8]:
filename[:-3]

'spec-56270-HD110609N070823M01_sp07-075.fits'

In [9]:
len(x)

52

In [90]:
for i in range(len(x)):
    
    url_lamost = x[i][0:-1]

    data_filename = url_lamost.split('/')[-1]
    
    
    for a in range(len(x[i][0:-1])):
        if x[i][0:-1][a:a+4] == 'spec':
            filename = x[i][0:-1][a:]
            if not os.path.exists(filename) :

                data = requests.get( url_lamost, stream=True)
                with open(data_filename, 'wb') as fdd:
                    for chunk in data.iter_content(chunk_size=1024): 
                        if chunk: # filter out keep-alive new chunks
                            fdd.write(chunk)

In [11]:
ptf1 = pyfits.open('spec-56270-HD110609N070823M01_sp07-075.fits')

In [12]:
ra = ptf1[0].header['RA']
dec = ptf1[0].header['Dec']

In [31]:
ra, dec

NameError: name 'ra' is not defined

In [30]:
all_target = pd.read_csv('epic_ha663_normal-hasub_new.csv')

In [14]:
for i in range(len(all_target['RA'])):
    if ra == all_target['RA'][i] and dec == all_target['Dec'][i]:
        print all_target['EPIC'][i]

201802777


In [23]:
for i in range(len(all_target['RA'])):
    if m.sqrt(((ra-all_target['RA'][i]))**2+((dec-all_target['Dec'][i]))**2) < 2./3600:
        print all_target['EPIC'][i]

NameError: name 'all_target' is not defined

In [28]:
os.chdir("D:\kepler_ptf_research\k2_c1")

In [29]:
os.getcwd()

'D:\\kepler_ptf_research\\k2_c1'

In [26]:
#column_names = ['EPIC','RA','Dec','GZ','FITS']  

#df = pd.DataFrame([ ], columns = column_names)
#df.to_csv('lamost_dr4.csv', index=False)

In [44]:
def append_csv(x1, x2, x3, x4, x5):

    df = pd.read_csv('lamost_dr4.csv')
    column_names =  ['EPIC','RA','Dec','GZ','FITS']
    
    if x4 not in list(df['GZ'])  :
    
    
        df2 = pd.DataFrame([[x1, x2, x3, x4, x5]], columns = column_names)

        df_add = df.append(df2)

        df_add.to_csv('lamost_dr4.csv', index = False)
    #else:
        #print x3,'already exists'

In [136]:
if '.gz' in 'spec-56270-HD110609N070823M01_sp07-075.fits.gz':
    print 'yes'
else:
    print 'no'

yes


In [167]:
x[0][0:-1].split('/')[-1]

'spec-56270-HD110609N070823M01_sp07-075.fits.gz'

In [174]:
x_gz_list = []

for i in range(len(x)):
    if '.gz' in x[i]:
        x_gz_list.append(i)

In [175]:
len(x_gz_list)

26

In [20]:
#最新版本

df = pd.read_csv('lamost_data1.csv')

def lamost_data_crawler(url):
    

    url_lamost = url[0:-1]
    #url_lamost = url
    data_filename = url_lamost.split('/')[-1]
    #data_filename = url.split('/')[-1]
    
    if data_filename not in list(df['GZ']) and '.gz' in data_filename:

        if not os.path.exists(data_filename):

            data = requests.get( url_lamost, stream=True)
            with open(data_filename, 'wb') as fdd:
                for chunk in data.iter_content(chunk_size=1024): 
                    if chunk: # filter out keep-alive new chunks
                        fdd.write(chunk)

        if '.gz' in data_filename and not os.path.exists(data_filename[:-3]):
            patoolib.extract_archive(data_filename)

            ptf1 = pyfits.open(data_filename[:-3])
            ra = ptf1[0].header['RA']
            dec = ptf1[0].header['Dec']

            for i in range(len(all_target['RA'])):
                if m.sqrt(((ra-all_target['RA'][i]))**2+((dec-all_target['Dec'][i]))**2) < 2./3600:

                    append_csv( all_target['EPIC'][i], all_target['RA'][i], all_target['Dec'][i], data_filename, data_filename[:-3])

                    if not os.path.exists('ktwo'+str(all_target['EPIC'][i])+'_C1'):
                        os.mkdir('ktwo'+str( all_target['EPIC'][i])+'_C1')

                    ptf1.close()

                    if not os.path.exists('D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/ktwo'+str(all_target['EPIC'][i])+'_C1/'+data_filename[:-3]):
                        os.rename(data_filename[:-3], 'D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/ktwo'+str(all_target['EPIC'][i])+'_C1/'+data_filename[:-3])
                    if not os.path.exists('D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/ktwo'+str(all_target['EPIC'][i])+'_C1/'+filename):
                        os.rename(data_filename, 'D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/ktwo'+str(all_target['EPIC'][i])+'_C1/'+data_filename)
    
    elif '.png' in data_filename:
        if not os.path.exists(data_filename):

            data = requests.get( url_lamost, stream=True)
            with open(data_filename, 'wb') as fdd:
                for chunk in data.iter_content(chunk_size=1024): 
                    if chunk: # filter out keep-alive new chunks
                        fdd.write(chunk)
        
    
        

In [21]:
for i in range(len(x)):
    lamost_data_crawler(x[i])

In [22]:
for i in range(len(x)):
    lamost_data_crawler(x[i])

In [7]:
df = pd.read_csv('lamost_data1.csv')

In [24]:
if 'spec-56283-HD111915N004324B01_sp04-083.fits.gz' not in list(df['GZ']):
    print 'no'
else:
    print 'It is in list'


It is in list


In [92]:
df['FITS']

0     spec-56270-HD110609N070823M01_sp07-075.fits
1     spec-56283-HD111915N004324B01_sp04-083.fits
2     spec-56283-HD111915N004324B01_sp15-144.fits
3     spec-56283-HD111915N004324M01_sp09-084.fits
4     spec-56283-HD111915N004324M01_sp11-049.fits
5     spec-56283-HD111915N004324M01_sp11-052.fits
6     spec-56283-HD111915N004324M01_sp12-097.fits
7     spec-56284-HD114225N022143B01_sp06-094.fits
8     spec-56284-HD114225N022143B01_sp09-064.fits
9     spec-56284-HD114225N022143B01_sp12-067.fits
10      spec-56286-HD114225N022143B_sp02-001.fits
11      spec-56286-HD114225N022143B_sp02-019.fits
12      spec-56286-HD114225N022143B_sp05-187.fits
13      spec-56286-HD114225N022143B_sp06-094.fits
14      spec-56286-HD114225N022143B_sp12-046.fits
15      spec-56286-HD114225N022143M_sp03-131.fits
16      spec-56286-HD114225N022143M_sp05-176.fits
17      spec-56286-HD114225N022143M_sp06-094.fits
18      spec-56286-HD114225N022143M_sp12-046.fits
19    spec-56287-HD112126N063805B01_sp02-092.fits


In [8]:
df['FITS'][0]

'spec-56270-HD110609N070823M01_sp07-075.fits'

In [107]:
df['FITS'][2][-13:-5]

'sp15-144'

In [43]:
df['FITS'][5][-13:-5]

'sp11-052'

In [125]:
for i in range(len(df['FITS'])):
    print df['FITS'][i]

spec-56270-HD110609N070823M01_sp07-075.fits
spec-56283-HD111915N004324B01_sp04-083.fits
spec-56283-HD111915N004324B01_sp15-144.fits
spec-56283-HD111915N004324M01_sp09-084.fits
spec-56283-HD111915N004324M01_sp11-049.fits
spec-56283-HD111915N004324M01_sp11-052.fits
spec-56283-HD111915N004324M01_sp12-097.fits
spec-56284-HD114225N022143B01_sp06-094.fits
spec-56284-HD114225N022143B01_sp09-064.fits
spec-56284-HD114225N022143B01_sp12-067.fits
spec-56286-HD114225N022143B_sp02-001.fits
spec-56286-HD114225N022143B_sp02-019.fits
spec-56286-HD114225N022143B_sp05-187.fits
spec-56286-HD114225N022143B_sp06-094.fits
spec-56286-HD114225N022143B_sp12-046.fits
spec-56286-HD114225N022143M_sp03-131.fits
spec-56286-HD114225N022143M_sp05-176.fits
spec-56286-HD114225N022143M_sp06-094.fits
spec-56286-HD114225N022143M_sp12-046.fits
spec-56287-HD112126N063805B01_sp02-092.fits
spec-56287-HD112126N063805B01_sp07-198.fits
spec-56287-HD112126N063805B01_sp08-162.fits
spec-56287-HD112126N063805M01_sp02-092.fits
spec-5

In [9]:
png_list = glob.glob('*png')

In [10]:
png_r_list = []
epic_r_list = []

for i in range(len(png_list)):
    if 'sp02-019' in png_list[i]:
        png_r_list.append(png_list[i])
        epic_r_list.append(df['EPIC'][i])

In [11]:
png_r_list

[]

In [110]:
str(int(epic_r_list[0]))

'201493524'

In [12]:
def mv_png(fits):
    png_r_list = []
    epic_r_list = []
    for i in range(len(png_list)):
        if fits[-13:-5] in png_list[i]:
            png_r_list.append(png_list[i])
            epic_r_list.append(df['EPIC'][i])
            
    os.chdir('ktwo'+str(int(epic_r_list[0]))+'_C1/')
    for a in range(len(png_r_list)):
        if not os.path.exists(png_r_list[a]): 
            os.chdir('D:\kepler_ptf_research\k2_c1\python_LAMOST_data_k2c1')
            
        else:
            os.chdir('D:\kepler_ptf_research\k2_c1\python_LAMOST_data_k2c1')
            
        if os.path.exists(png_r_list[a]):
            os.rename(png_r_list[a], 'ktwo'+str(int(epic_r_list[0]))+'_C1/'+png_r_list[a])
        #else:
            #os.chdir('..')
        
            

In [13]:
for s in range(len(df['FITS'])):
    mv_png(df['FITS'][s])

IndexError: list index out of range

In [14]:
os.getcwd()

'D:\\kepler_ptf_research\\k2_c1\\python_LAMOST_data_k2c1'

In [195]:
os.chdir('D:\kepler_ptf_research\k2_c1\python_LAMOST_data_k2c1')

In [18]:
list_epic = df['EPIC']

In [32]:
os.chdir('D:\kepler_ptf_research\k2_c1\python_LAMOST_data_k2c1\DR4/')

In [47]:
gz_list = glob.glob('*gz')

In [48]:
len(gz_list)

533

In [37]:
gz_list[0][0:-3]

'spec-55918-F5591807_sp01-012.fits'

In [45]:
df = pd.read_csv('lamost_dr4.csv')

def lamost_data_crawler(gzname):
    


    if '.gz' in gzname:
        patoolib.extract_archive(gzname)

        ptf1 = pyfits.open(gzname[:-3])
        ra = ptf1[0].header['RA']
        dec = ptf1[0].header['Dec']
        
        ptf1.close()
        
        for i in range(len(all_target['RA'])):
            if m.sqrt(((ra-all_target['RA'][i]))**2+((dec-all_target['Dec'][i]))**2) < 2./3600:

                append_csv( all_target['EPIC'][i], all_target['RA'][i], all_target['Dec'][i], gzname, gzname[:-3])

                if not os.path.exists('ktwo'+str(all_target['EPIC'][i])+'_C1'):
                    os.mkdir('ktwo'+str( all_target['EPIC'][i])+'_C1')

                

                if not os.path.exists('D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/DR4/ktwo'+str(all_target['EPIC'][i])+'_C1/'+gzname[:-3]):
                    os.rename(gzname[:-3], 'D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/DR4/ktwo'+str(all_target['EPIC'][i])+'_C1/'+gzname[:-3])
                if not os.path.exists('D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/DR4/ktwo'+str(all_target['EPIC'][i])+'_C1/'+gzname):
                    os.rename(gzname, 'D:/kepler_ptf_research/k2_c1/python_LAMOST_data_k2c1/DR4/ktwo'+str(all_target['EPIC'][i])+'_C1/'+gzname)

 



In [49]:
for i in range(len(gz_list)):
    lamost_data_crawler(gz_list[i])

patool: Extracting spec-55918-F5591807_sp01-012.fits.gz ...
patool: ... spec-55918-F5591807_sp01-012.fits.gz extracted to `spec-55918-F5591807_sp01-012.fits'.
patool: Extracting spec-55918-F5591807_sp01-212.fits.gz ...
patool: ... spec-55918-F5591807_sp01-212.fits.gz extracted to `spec-55918-F5591807_sp01-212.fits'.
patool: Extracting spec-55918-F5591807_sp14-111.fits.gz ...
patool: ... spec-55918-F5591807_sp14-111.fits.gz extracted to `spec-55918-F5591807_sp14-111.fits'.
patool: Extracting spec-55973-F5597306_sp02-094.fits.gz ...
patool: ... spec-55973-F5597306_sp02-094.fits.gz extracted to `spec-55973-F5597306_sp02-094.fits'.
patool: Extracting spec-55973-F5597306_sp10-083.fits.gz ...
patool: ... spec-55973-F5597306_sp10-083.fits.gz extracted to `spec-55973-F5597306_sp10-083.fits'.
patool: Extracting spec-55973-F5597306_sp10-104.fits.gz ...
patool: ... spec-55973-F5597306_sp10-104.fits.gz extracted to `spec-55973-F5597306_sp10-104.fits'.
patool: Extracting spec-55977-F5597703_sp01-09

In [50]:
ktwo_list = glob.glob('ktwo*')

In [51]:
len(ktwo_list)

386

In [52]:
df4 = pd.read_csv('lamost_dr4.csv')

In [54]:
len(df4['FITS'])

533

In [62]:
df4['FITS'][0][-13:-5]

'sp01-012'

In [55]:
epic_list = list(df4['EPIC'])

In [56]:
png_list = glob.glob('*png')

In [66]:
len(glob.glob('*png'))

533

In [67]:
def mv_png(fits):
    png_r_list = []
    epic_r_list = []
    for i in range(len(png_list)):
        if fits[-13:-5] in png_list[i]:
            png_r_list.append(png_list[i])
            epic_r_list.append(df4['EPIC'][i])
            
    os.chdir('ktwo'+str(int(epic_r_list[0]))+'_C1/')
    for a in range(len(png_r_list)):
        if not os.path.exists(png_r_list[a]): 
            os.chdir('D:\kepler_ptf_research\k2_c1\python_LAMOST_data_k2c1\DR4')
            
        else:
            os.chdir('D:\kepler_ptf_research\k2_c1\python_LAMOST_data_k2c1\DR4')
            
        if os.path.exists(png_r_list[a]):
            os.rename(png_r_list[a], 'ktwo'+str(int(epic_r_list[0]))+'_C1/'+png_r_list[a])
        #else:
            #os.chdir('..')
        

In [68]:
for i in range(len(df4['FITS'])):
    mv_png(df4['FITS'][i])